In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

In [2]:
feature_number = 105

In [3]:
X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [4]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.fit_transform(X)

In [5]:
pca = PCA(n_components= 0.999).fit(X_scaled)
X_pca = pca.transform(X_scaled)
print(X_pca.shape)

(393345, 82)


In [6]:
model_X = X_pca
model_y = y
print(model_X.shape, model_y.shape)

(393345, 82) (393345,)


In [7]:
# find how many kinds of values in y
y_values = np.unique(model_y)
print(y_values)

[0. 1. 2. 3. 4. 5. 6.]


In [8]:
from cnn_builder2 import *


# learning_rate = 2e-2
# channel = 32


# model_X_normalized = torch.Tensor(model_X)
# model_X_normalized = torch.unsqueeze(model_X_normalized, 1)


# cnn = CNN_Sleeping(channels = channel)

# model = skorch.NeuralNetClassifier(cnn, criterion=torch.nn.CrossEntropyLoss,
#                              device="cuda",
#                              optimizer=torch.optim.SGD ,
#                              lr=learning_rate,
#                              max_epochs=120,
#                              batch_size=128,
#                              callbacks=[skorch.callbacks.EarlyStopping(lower_is_better=True)])

# model.fit(model_X_normalized, np.asarray(model_y, dtype=np.int64))

train_cnn_model(model_X, model_y)

Start with learning rate: 0.01 and channel: 128
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.8974       0.8274        0.5809  20.8858
      2        0.4676       0.8701        0.4123  21.1614
      3        0.3783       0.8833        0.3484  21.3343
      4        0.3344       0.8917        0.3147  21.4478
      5        0.3094       0.8973        0.2954  21.5084
      6        0.2941       0.9009        0.2835  21.4607
      7        0.2838       0.9042        0.2752  21.4971
      8        0.2761       0.9063        0.2689  22.3279
      9        0.2698       0.9074        0.2641  22.6659
     10        0.2646       0.9091        0.2597  23.0771
     11        0.2600       0.9098        0.2560  23.2140
     12        0.2558       0.9114        0.2528  22.9663
     13        0.2521       0.9120        0.2500  23.0044
     14        0.2487       0.9129        0.2474  23.0213
     15        0.2455   

In [9]:
# distribution of labels y
# print every count of labels
for i in y_values:
    print(i, np.sum(model_y == i))


0.0 281216
1.0 15578
2.0 62754
3.0 5255
4.0 3231
5.0 24275
6.0 1036
